**1. Data Preprocessing**

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

In [16]:
from google.colab import files
uploaded = files.upload()

Saving anime.csv to anime.csv


In [27]:
anime_data = pd.read_csv('anime.csv')
anime_data

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [28]:
anime_data.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [29]:
anime_data.tail()

,anime_id,name,genre,type,episodes,rating,members
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175
12293,26081,Yasuji no Pornorama: Yacchimae!!,Hentai,Movie,1,5.46,142


In [30]:
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [31]:
anime_data.size

86058

In [32]:
anime_data.shape

(12294, 7)

In [33]:
anime_data.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


In [34]:
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [35]:
# Check for missing values
print(anime_data.isnull().sum())

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64


In [38]:
# For simplicity, we can drop rows with missing values
anime_data.dropna(inplace=True)

In [39]:
# Verify that there are no missing values left
print(anime_data.isnull().sum())

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64


**2.  Feature Extraction**

In [42]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [41]:
# Convert categorical features to numerical
label_encoder = LabelEncoder()
anime_data['genre_encoded'] = label_encoder.fit_transform(anime_data['genre'])

In [43]:
# Normalize numerical features
scaler = MinMaxScaler()
anime_data[['rating', 'members']] = scaler.fit_transform(anime_data[['rating', 'members']])

**3. Recommendation System**

In [44]:
from sklearn.metrics.pairwise import cosine_similarity

In [46]:
# Compute cosine similarity matrix
features = anime_data[['genre_encoded', 'rating', 'members']]
cosine_sim_matrix = cosine_similarity(features)

In [54]:
# Function to recommend anime
def recommend_anime(anime_title, cosine_sim_matrix, anime_df, top_n=10):
    # Get the index of the anime that matches the title
    idx = anime_df[anime_df['title'] == anime_title].index[0]

    # Get the pairwise similarity scores of all animes with that anime
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))

    # Sort the anime based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top_n most similar anime
    sim_scores = sim_scores[1:top_n+1]

    # Get the anime indices
    anime_indices = [i[0] for i in sim_scores]

    # Return the top_n most similar anime
    return anime_df['title'].iloc[anime_indices]

**4. Evaluation**

In [56]:
from sklearn.model_selection import train_test_split

# Split the dataset
train_df, test_df = train_test_split(anime_data, test_size=0.2, random_state=42)

In [57]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [58]:
# Function to evaluate the recommendation system
def evaluate_recommendation_system(test_df, cosine_sim_matrix, anime_df, top_n=10):
    y_true = []
    y_pred = []

    for idx, row in test_df.iterrows():
        # Get the true list of similar anime
        true_anime = test_df[test_df['genre'] == row['genre']]['title'].tolist()

        # Get the predicted list of similar anime
        pred_anime = recommend_anime(row['title'], cosine_sim_matrix, anime_df, top_n)

        # Update y_true and y_pred lists
        y_true.append(true_anime)
        y_pred.append(pred_anime)

    # Flatten the lists
    y_true = [item for sublist in y_true for item in sublist]
    y_pred = [item for sublist in y_pred for item in sublist]

    # Calculate precision, recall, and F1-score
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    return precision, recall, f1

In [ ]:
# Example usage
precision, recall, f1 = evaluate_recommendation_system(test_df, cosine_sim_matrix, anime_data)
print(f'Precision: {precision}, Recall: {recall}, F1-score: {f1}')